TEAM 10: Eliecer Diaz

AIM: 

Dataset of validated Chest X-Ray images described and analyzed in "Deep learning-based classification and referral of treatable human diseases".
The data was split into three subsets: - 60 % train - 20 % validation - 20 % test
Kermany, Daniel; Zhang, Kang; Goldbaum, Michael (2018), “Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification”, Mendeley Data, v2
Version 2, published 2018-01-06 University of California San Diego

In [1]:
import random 
import collections
random.seed(7)
import tensorflow as tf
from tensorflow.keras import layers,models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import SensitivityAtSpecificity
import matplotlib.pyplot as plt
from time import time
import tensorflow.keras.backend as k
import tensorflow.keras.callbacks as Callback
import matplotlib.pyplot as plot
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print("Found GPU at: {}".format(device_name))


Found GPU at: /device:GPU:0


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [3]:
train_dir = r'/kaggle/input/pneumonia/pneumonia2/train'
val_dir = r'/kaggle/input/pneumonia/pneumonia2/validation'
test_dir = r'/kaggle/input/pneumonia/pneumonia2/test'

In [4]:
os.listdir(train_dir)

['NORMAL', 'PNEUMONIA']

In [5]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_generator = val_datagen.flow_from_directory(
        test_dir,
        #verbose = 0,
        shuffle=False,
        target_size = (150, 150),
        batch_size = 1, #<----tensorflow documentation
        class_mode='binary')

Found 1171 images belonging to 2 classes.


In [6]:
def model_(SS, batch): # SS: number neurons in the last dense layer, batch: batch size 
    
    #getting new metrics to calculate F1 at the end of the notebook
    Preci = tf.keras.metrics.Precision() 
    Recal = tf.keras.metrics.Recall() 
    
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range = 5, 
                                       horizontal_flip = True)
    
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        #verbose = 0,
        target_size = (150, 150),
        batch_size = batch,
        class_mode = 'binary')
    
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        #verbose = 0,
        target_size = (150, 150),
        batch_size = batch,
        class_mode='binary')
    
    try:
        with tf.device('/device:GPU:0'):
        #with strategy.scope():
            model = tf.keras.models.Sequential()
            model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(tf.keras.layers.Flatten())
            model.add(Dense(SS, activation = 'relu'))
            model.add(Dense(1, activation = 'sigmoid'))

            model.compile(loss = 'binary_crossentropy',
                          optimizer = tf.keras.optimizers.RMSprop(lr = 1e-4),
                          metrics = ['acc', Preci, Recal])
            print("GPU/TPU IS ON")
    except:
        
            model = tf.keras.models.Sequential()
            model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(Conv2D(32, (3, 3), activation = 'relu'))
            model.add(MaxPooling2D((2, 2)))
            model.add(tf.keras.layers.Flatten())
            model.add(Dense(SS, activation = 'relu'))
            model.add(Dense(1, activation = 'sigmoid'))

            model.compile(loss = 'binary_crossentropy',
                          optimizer = tf.keras.optimizers.RMSprop(lr = 1e-4),
                          metrics = ['acc', Preci, Recal])
            print("NOT USING GPU")

    return model, train_generator, val_generator

In [7]:
SS = [512] # we divide them by 10 after to get: 0.5, 0.7, 0.9
batches = [8, 32, 64]
models = {}
train_generators = {}
val_generators = {}

for ss in SS:
    for batch in batches:
        model_ss_batch = 'models_{}_{}'.format(ss, batch)  # creating the keys
        models[model_ss_batch] = model_(ss//10, batch) # appending the model to the key
        
        train_generator_ss_batch = 'train_generators_{}_{}'.format(ss, batch)  # creating the keys
        train_generators[train_generator_ss_batch] = model_(ss//10, batch) # appending the generator to the key
        
        val_generator_ss_batch = 'val_generators_{}_{}'.format(ss, batch)  # creating the keys
        val_generators[val_generator_ss_batch] = model_(ss//10, batch) # appending the model to the key       
        

Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON
Found 3513 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
GPU/TPU IS ON


In [8]:
print(models.keys()) #prints keys
print(train_generators.keys()) #prints keys
print(val_generators.keys()) #prints keys

print(test_generator.n)

dict_keys(['models_512_8', 'models_512_32', 'models_512_64'])
dict_keys(['train_generators_512_8', 'train_generators_512_32', 'train_generators_512_64'])
dict_keys(['val_generators_512_8', 'val_generators_512_32', 'val_generators_512_64'])
1171


**Model Fitting**

In [10]:
Final_models = {}
for i in range(len(models)):
    print('MODEL:',i)
    Final_model_i = 'models_{}'.format(i)  # creating the keys
    Final_models[Final_model_i] = models[list(models.keys())[i]][0].fit_generator(
        train_generators[list(train_generators.keys())[i]][1],
        steps_per_epoch = None, #3513//batches[i],#None, # = 3513 / 8
        verbose = 0,
        epochs = 100,
        validation_data = val_generators[list(val_generators.keys())[i]][1],
        validation_steps = None, #1172//batches[i] #None # = 1172 / 8
    )
    # saving the model
    models[list(models.keys())[i]][0].save("model{}.h5".format(i))

MODEL: 0


KeyboardInterrupt: 

In [ ]:
for i in range(len(Final_models)):
    print(i)
    models_key = Final_models.keys()
    model_key = [j  for  j in  models_key]
    metrics = Final_models[list(Final_models.keys())[i]].history.keys()
    metric = [k  for  k in  metrics]
    
    acc = Final_models[list(Final_models.keys())[i]].history[metric[1]]
    val_acc = Final_models[list(Final_models.keys())[i]].history[metric[5]]
    
    preci = Final_models[list(Final_models.keys())[i]].history[metric[2]]
    val_preci = Final_models[list(Final_models.keys())[i]].history[metric[6]]
    
    recal = Final_models[list(Final_models.keys())[i]].history[metric[3]]
    val_recal = Final_models[list(Final_models.keys())[i]].history[metric[7]]
    
    loss = Final_models[list(Final_models.keys())[i]].history[metric[0]]
    val_loss = Final_models[list(Final_models.keys())[i]].history[metric[4]]
    
    epochs = range(len(acc))
        
    fig, axs = plt.subplots(1, 4, figsize=(10, 10))
    
    
    axs[ 0].set_title('1. Accuracy')
    axs[ 0].plot(epochs, acc, 'bo-', label = 'Training acc')
    axs[ 0].plot(epochs, val_acc, 'r*-', label = 'Validation acc')
    
    axs[ 1].set_title('2. Precision ')
    axs[ 1].plot(epochs, preci, 'bo-', label = 'Precision_training ')
    axs[ 1].plot(epochs, val_preci, 'r*-', label = 'Precision validation')
    
    axs[ 2].set_title('3. Recall (sensitivity) ')
    axs[ 2].plot(epochs, recal, 'bo-', label = 'Recall training')
    axs[ 2].plot(epochs, val_recal, 'r*-', label = 'Recall validation')
        
    axs[ 3].set_title('4. Loss')
    axs[ 3].plot(epochs, loss, 'bo-', label = 'Training loss')
    axs[ 3].plot(epochs, val_loss, 'r*-', label = 'Validation loss')
    
      
    fig.suptitle(model_key[i], fontname="Times New Roman",fontweight="bold")
    fig.text(0.5, 0.04, 'EPOCH', ha='center', fontname="Times New Roman",fontweight="bold")
    plt.show()

In [ ]:
# RESULTS? YES...
test_generator.reset() # resetting generator
for i in range(0,len(Final_models)):
    Loss, Accuracy, Preci, Recal = models[list(models.keys())[i]][0].evaluate_generator(generator=val_generators[list(val_generators.keys())[i]][1], steps=10)
    print("Model",i)
    print('Loss: {}'.format(Loss), 'Accuracy: {}'.format(Accuracy), 'Precision: {}'.format(Preci), 'Recall: {}'.format(Recal))

In [ ]:
# Get actuall y_labels from test set
Y_labels = test_generator.classes

# E.g. using model 2, make some predictions with cut-off 0.5
for i in range(0,len(Final_models)):
    print("Prediction for model {}:".format(i))
    
    Y_pred = models[list(models.keys())[i]][0].predict_generator(test_generator)
    Y_pred = 1*(Y_pred.astype('float64') > 0.5)
    
    print('Confusion Matrix')
    print(confusion_matrix(Y_labels, Y_pred))
    print('Classification Report')
    target_names = ['1','0']
    print(classification_report(Y_labels, Y_pred, target_names=target_names))
    

In [ ]:
logit_roc_auc = roc_auc_score(Y_labels, Y_pred)
#logit_roc_auc_small = roc_auc_score(y_test, model.predict(X_test))
#logit_roc_auc_small_reg1 = roc_auc_score(y_test, model.predict(X_test))

fpr, tpr, thresholds = roc_curve(Y_labels, Y_pred)
#fpr_small, tpr_small, thresholds_small = roc_curve(y_test, model_small.predict_proba(X_test))
#fpr_small_reg1, tpr_small_reg1, thresholds_small_reg1 = roc_curve(y_test, model_small_reg1.predict_proba(X_test))
plt.figure()
plt.plot(fpr, tpr, label='MODEL 1 (area = %0.2f)' % logit_roc_auc)
#plt.plot(fpr_small, tpr_small, label='MODEL SMALL (area = %0.2f)' % logit_roc_auc_small)
#plt.plot(fpr_small_reg1, tpr_small_reg1, label='MODEL SMALL + REGUL2 (area = %0.2f)' % logit_roc_auc_small_reg1)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


In [ ]:
# What for??
#sensitivity at specificity? sensitivity 
Y_labels = Y_labels.reshape(1171,)
Y_pred = Y_pred.reshape(1171,)
m = tf.keras.metrics.SensitivityAtSpecificity(0.9, num_thresholds = 1)
m.update_state(Y_labels, Y_pred)
m.result().numpy()